<a href="https://colab.research.google.com/github/FelixOgero/Analysis-of-Company-Stocks/blob/main/StockPredictionProgram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install quandl

In [2]:
# This program predicts stock prices using machine learning models
import quandl
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

In [4]:
# Get the stock data
df = quandl.get("WIKI/FB")
print(df.head())

             Open   High    Low    Close       Volume  Ex-Dividend  \
Date                                                                 
2012-05-18  42.05  45.00  38.00  38.2318  573576400.0          0.0   
2012-05-21  36.53  36.66  33.00  34.0300  168192700.0          0.0   
2012-05-22  32.61  33.59  30.94  31.0000  101786600.0          0.0   
2012-05-23  31.37  32.50  31.36  32.0000   73600000.0          0.0   
2012-05-24  32.95  33.21  31.77  33.0300   50237200.0          0.0   

            Split Ratio  Adj. Open  Adj. High  Adj. Low  Adj. Close  \
Date                                                                  
2012-05-18          1.0      42.05      45.00     38.00     38.2318   
2012-05-21          1.0      36.53      36.66     33.00     34.0300   
2012-05-22          1.0      32.61      33.59     30.94     31.0000   
2012-05-23          1.0      31.37      32.50     31.36     32.0000   
2012-05-24          1.0      32.95      33.21     31.77     33.0300   

           

In [5]:
# Get the Adjusted close price
df = df[['Adj. Close']]
print(df.head())

            Adj. Close
Date                  
2012-05-18     38.2318
2012-05-21     34.0300
2012-05-22     31.0000
2012-05-23     32.0000
2012-05-24     33.0300


In [9]:
# A variable for predicting "n" days out into the future
forecast_out = 30
# Create another column (the target or dependent variable) shifted "n" units up
df['Prediction'] = df[['Adj. Close']].shift(-forecast_out)
print(df.tail())

            Adj. Close  Prediction
Date                              
2018-03-21      169.39         NaN
2018-03-22      164.89         NaN
2018-03-23      159.39         NaN
2018-03-26      160.06         NaN
2018-03-27      152.19         NaN


In [10]:
# Create the independent dataset X
# Convert the dataframe to a numpy array
X = np.array(df.drop(['Prediction'], 1))
# Remove the last "n" rows
X = X[:-forecast_out]
print(X)

[[ 38.2318]
 [ 34.03  ]
 [ 31.    ]
 ...
 [171.5499]
 [175.98  ]
 [176.41  ]]


<ipython-input-10-ba6d89afc1c1>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = np.array(df.drop(['Prediction'], 1))


In [11]:
# Create the dependent dataset y
y = np.array(df['Prediction'])
y = y[:-forecast_out]
print(y)

[ 30.771  31.2    31.47  ... 159.39  160.06  152.19 ]


In [12]:
# Split the data into .8 training and .2 testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [13]:
# Craete and train the support vector machine (Regressor)
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr_rbf.fit(X_train, y_train)

SVR(C=1000.0, gamma=0.1)

In [14]:
# Testing model: Score returns the coefficient of determination R^2 of the prediction
svm_confidence = svr_rbf.score(X_test, y_test)
print("svm confidence: ", svm_confidence)


svm confidence:  0.9808511714023465


In [15]:
# Create and train the linear regression model
lr = LinearRegression()
# Train the model
lr.fit(X_train, y_train)

LinearRegression()

In [18]:
# Testing model: Score returns the coefficient of determination R^2 of the prediction
lr_confidence = lr.score(X_test, y_test)
print("lr confidence: ", lr_confidence)

lr confidence:  0.978818504533401


In [19]:
# Set x_forecast equal to the last 30 rows of the original data set from Adj. Close column
x_forecast = np.array(df.drop(['Prediction'], 1))[-forecast_out:]
print(x_forecast)

[[173.15]
 [179.52]
 [179.96]
 [177.36]
 [176.01]
 [177.91]
 [178.99]
 [183.29]
 [184.93]
 [181.46]
 [178.32]
 [175.94]
 [176.62]
 [180.4 ]
 [179.78]
 [183.71]
 [182.34]
 [185.23]
 [184.76]
 [181.88]
 [184.19]
 [183.86]
 [185.09]
 [172.56]
 [168.15]
 [169.39]
 [164.89]
 [159.39]
 [160.06]
 [152.19]]


<ipython-input-19-3069d41948c3>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  x_forecast = np.array(df.drop(['Prediction'], 1))[-forecast_out:]


In [22]:
# Print the linear regression predictions for the next "n" days
lr_prediction = lr.predict(x_forecast)
print(lr_prediction)

# Print the support vector machine predictions for the next "n" days
svm_prediction = svr_rbf.predict(x_forecast)
print(svm_prediction)

[177.47844874 183.93854107 184.38476409 181.74799171 180.37889836
 182.30577048 183.40104517 187.76186103 189.42505591 185.90597893
 182.72156921 180.30790833 180.99752572 184.83098711 184.20221831
 188.18780118 186.79842497 189.72929888 189.25265157 186.33191908
 188.67458993 188.33992267 189.58731883 176.88010424 172.40773262
 173.66527022 169.10162571 163.52383798 164.20331394 156.22200677]
[181.98245441 181.41348194 181.38792508 175.34816881 176.09895272
 177.1777271  180.86598884 186.37009527 179.49835486 184.27754541
 178.8224536  176.32193291 174.78864637 181.4639239  181.41811515
 184.23581949 187.52833964 179.63524032 179.68329441 186.07400517
 181.62876308 183.39157634 179.5008265  180.68151151 171.58898932
 172.05190349 172.16153734 167.56299272 166.26410647 163.70103073]
